In [1]:
import sys
import os
import re
sys.path.append('../')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import time
sns.set()
pd.set_option('display.max_columns', None)
pd.option_context("display.max_rows", None)
import warnings
warnings.filterwarnings("ignore")
from fancyimpute import KNN
import joblib
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

## Data preparation

In [4]:
df = pd.read_excel("../01_Data/01_Raw/DATASET_TEST_v2_No_Premium.xlsx")
ID = df.SUBMISSION_ID
df.head()

,SUBMISSION_ID,DIVISION_ID,FACUL_NUM,TAG,DIVISION_NUM,SEGMENT_LOB,UF_STATUS,CT_STATUS,UWYEAR,INCEPTION_MONTH,INCEPTION,EXPIRY,CT_PERIOD,MARKET,SUBSECTOR,MAINOCCUPANCY,SECTOR,BUSINESSUNIT,UWCENTER,SCOPE_PERILS,SUBSIDIARY,COVERAGE,PARTTYPE,MAIN_COUNTRY,GUARANTEE,MAIN_PRICING_CATEG,MAIN_PRICING_SUBCATEG,BI_TYPE,BI_PERIOD,TC_PROFIT_COMMISSION,DEDUCTIBLES,INSUREDVALUEPD,INSUREDVALUEBI,TOTALINSUREDVALUE,NBLOCS,OIL,LIMIT,ATTACHMENT,SCOR_SHARE,MODELED_CAT_EXPLOSS,DISCOUNTS,DEDUCTION,EXT_EXPENSE,WORDING,QUALITY_RISK_MGT,ASSET_QUALITY,BI_MITIGATION,MB_QUALITY,REPORTCCY,ORIGCCY,TXCHANGE,PRICING_DATE,FXRATEUSD,TOP_MPL,TOP_FMLS
0,1,946765,FA0020462,Bound,1,Ppty Non Energy,Active,Finalized,2020-1,JUNE,2020-06-30,2021-06-30,12,CANADA,MINING COMPANY,Precious Metals Mines,BS Energy,BS Energy Mining,CANADA,All Risks,SCOR CANADA,"PD,BI",Quota-Share,Argentina,Unspecified,Mining - Hard Rock,Standard,Loss of profit,12.0,0.0,"PD: ,5M, BI: 30Day(s)",6.386765e+08,3.094686e+08,9.481450e+08,3,0.0,180635834.0,0.0,0.025,0.0,0.0,0.072997,0.072997,Standard,Average,Average,Average,NaN,EUR,USD,0.903179,2020-06-30,1.0,2.500444e+08,NaN
1,2,866600,11F008861,Signed-off,1,Ppty Non Energy,Active,Finalized,2019-1,JUNE,2019-06-30,2020-06-30,12,CANADA,MINING COMPANY,Precious Metals Mines,BS Energy,BS Energy Mining,CANADA,All Risks,SCOR CANADA,"PD,BI",Quota-Share,Argentina,Unspecified,Mining - Hard Rock,Standard,Loss of profit,12.0,0.0,"PD: ,4M, BI: 30Day(s)",6.120514e+08,1.912141e+08,8.032655e+08,3,0.0,175963400.0,0.0,0.075,0.0,0.0,0.093420,0.093420,Standard,Average,Average,Average,NaN,EUR,USD,0.879817,2019-07-02,1.0,2.097005e+08,NaN
2,3,708414,FA0003626,Bound,1,Ppty Non Energy,Terminated,Cancelled,2017-1,OCTOBER,2017-10-02,2018-10-01,12,ARGENTINA,MINING COMPANY,Precious Metals Mines,BS Energy,BS Energy Mining,LATIN AMERICA,All Risks,SCOR RE US,"PD,BI",Excess of Loss,Argentina,Unspecified,Mining - Hard Rock,Standard,Loss of profit,24.0,0.0,"PD: 4,6M, BI: 30Day(s)",1.585276e+09,5.624312e+08,2.147707e+09,1,0.0,278732700.0,0.0,0.075,0.0,0.0,0.095564,0.095564,Standard,Average,Average,Average,NaN,EUR,USD,0.929109,2018-03-05,1.0,8.826536e+08,NaN
3,4,754122,11F007069,Bound,2,Ppty Non Energy,Terminated,Cancelled,2018-1,MAY,2018-05-31,2019-05-31,12,Worldwide,MINING COMPANY,Precious Metals Mines,BS Energy,BS Energy Mining,CANADA,All Risks,SCOR CANADA,"PD,BI",Quota-Share,Argentina,Unspecified,Mining - Hard Rock,Standard,Loss of profit,24.0,0.0,"PD: 2,1M, BI: 30Day(s)",8.259934e+08,9.739950e+08,1.799988e+09,2,0.0,589622642.0,0.0,0.120,0.0,0.0,0.635696,0.635696,Standard,Average,Average,Average,NaN,EUR,USD,0.842318,2019-02-06,1.0,9.285377e+08,NaN
4,5,946759,11F008861,Bound,1,Ppty Non Energy,Active,Finalized,2020-1,JUNE,2020-06-30,2021-06-30,12,CANADA,MINING COMPANY,Precious Metals Mines,BS Energy,BS Energy Mining,CANADA,All Risks,SCOR CANADA,"PD,BI",Quota-Share,Argentina,Unspecified,Mining - Hard Rock,Standard,Loss of profit,12.0,0.0,"PD: ,5M, BI: 30Day(s)",6.387283e+08,3.094292e+08,9.481575e+08,3,0.0,180635834.0,0.0,0.075,0.0,0.0,0.080394,0.080394,Standard,Average,Average,Average,NaN,EUR,USD,0.903179,2020-06-30,1.0,2.500452e+08,NaN


In [5]:
df.drop(['SUBMISSION_ID', 'DIVISION_ID','FACUL_NUM','UWYEAR','INCEPTION_MONTH','EXPIRY',
         'TOP_FMLS', 'MB_QUALITY','MAIN_COUNTRY', 'REPORTCCY','ORIGCCY'], axis=1, inplace=True)

In [6]:
df[['a', 'b']] = df.DEDUCTIBLES.str.split(', ',expand=True)
df['pd_bi'] = ""
df['pd'] = ""
df['bi'] = ""

In [7]:
for i, value in enumerate(df.a):
    try:
        if value[:6] == 'PD,BI:':
            df['pd_bi'][i] = value[6:]
            df['pd'][i] = np.nan
        elif value[:3] == 'PD:' and value[9:12] != 'BI:':
            df['pd_bi'][i] = np.nan
            df['pd'][i] = value[3:]
    except Exception: # pass NaNs
        pass 
    
for i, value in enumerate(df.b):
    try:
        df['bi'][i] = value[4:]
    except Exception: # pass nans
        df['bi'][i] = np.nan

In [8]:
df.drop(['a','b'], axis=1, inplace=True)

In [9]:
# clean PD_BI
for i, value in enumerate(df['pd_bi']):
    try:
        df['pd_bi'][i] = df['pd_bi'][i].replace(',','.')
        df['pd_bi'][i] = df['pd_bi'][i].replace('M','')
        df['pd_bi'] = df['pd_bi'].apply(pd.to_numeric)
    except Exception:
        pass

In [10]:
for i, value in enumerate(df['pd']):
    try:
        if 'M' in value:
            df['pd'][i] = df['pd'][i].replace(' ' and 'M', '')
            df['pd'][i] = df['pd'][i].replace(',','.')
        elif 'loss' in value:
            df['pd'][i] = df['pd'][i].replace('% of loss' or '%ofloss', '')
            df['pd'][i] = df['pd'][i].replace(',','.')
            df['pd'][i] = pd.to_numeric(df['pd'][i])
            df['pd'][i] = df['pd'][i] / 100
        elif 'tiv' in value:
            df['pd'][i] = df['pd'][i].replace('% of tiv' or '%oftiv', '')
            df['pd'][i] = df['pd'][i].replace(',','.')
            df['pd'][i] = pd.to_numeric(df['pd'][i])
            df['pd'][i] = df['pd'][i] / 100  
    except Exception as e:
        pass
df['pd'] = df['pd'].apply(pd.to_numeric)

In [11]:
# clean BI
for i, value in enumerate(df['bi']):
    try:
        df['bi'][i] = df['bi'][i].replace('M', '')
        df['bi'][i] = df['bi'][i].replace('Day(s)' and '% of loss', '')
        df['bi'][i] = df['bi'][i].replace('Day(s)' or '% of loss', '')
        df['bi'][i] = df['bi'][i].replace(',','.')
    except Exception as e:
        pass

In [12]:
df['bi'] = df['bi'].apply(pd.to_numeric)
df.drop(['DEDUCTIBLES'], axis=1, inplace=True)

In [13]:
df['inception_year'], df['inception_month'] = df['INCEPTION'].dt.year, df['INCEPTION'].dt.month
df['inception_duration'], df['pricing_duration'] = datetime.now() - df['INCEPTION'], datetime.now() - df['PRICING_DATE']
df['inception_duration'], df['pricing_duration'] = df['inception_duration'].dt.days, df['pricing_duration'].dt.days

In [14]:
df.drop(['INCEPTION', 'PRICING_DATE','INSUREDVALUEPD','EXT_EXPENSE','TXCHANGE'], axis=1, inplace=True)

In [15]:
# if not below, consider as one category
for i, value in enumerate(df.MAINOCCUPANCY):
    if value not in ['Petrochemical (Major agents)','Independants','Multi-occupancy industrial','Multi-Activity Mining','Coal mine underground']:
        df.MAINOCCUPANCY[i] = 'Other'
    else:
        pass

In [16]:
for i, value in enumerate(df.MARKET):
    if value not in ['USA','BAHRAIN','UNITED ARAB EMIR','Worldwide']:
        df.MARKET[i] = 'Other'
    else:
        pass

In [17]:
for i, value in enumerate(df.MAIN_PRICING_SUBCATEG):
    if value not in ['Aggravated','Medium: VCM/ BTX/ polymers/ styrene/ ethylbenzene/ EG?',
                     'Compressor/ Pumping Station','Steel plant with direct reduction',
                     'Storage Terminal','Combined cycle gas turb','Chocolate industry']:
        df.MAIN_PRICING_SUBCATEG[i] = 'Other'
    else:
        pass

In [18]:
df = pd.get_dummies(df)
df.shape

(3259, 174)

## Predict on test set

In [19]:
df.drop(['GUARANTEE_Windstorm', 'MAIN_PRICING_CATEG_Railway manufacturing industry',
         'SUBSECTOR_SHIPBUILDING COMPANY'], axis=1, inplace=True)

In [20]:
xgb = joblib.load('xgb_reg.pkl')
df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df.columns.values]
y_pred_xgb = xgb.predict(df)

In [21]:
y_pred_df = pd.DataFrame(y_pred_xgb, columns=['predicted_premium'])
result = pd.concat([ID, y_pred_df], names=['SUBMISSION_ID','predicted_premium'], axis=1)

In [30]:
from scipy.special import inv_boxcox
result['predicted_premium'] = inv_boxcox(result['predicted_premium'], 0.17700863235182274)

In [31]:
result['predicted_premium']

0        81095.679688
1       105976.203125
2       177839.093750
3       156040.828125
4       161765.984375
            ...      
3254     39137.675781
3255     39072.113281
3256      6113.314941
3257    196980.500000
3258     21869.656250
Name: predicted_premium, Length: 3259, dtype: float32

In [32]:
result.to_csv("../01_Data/02_Results/result_premium2.csv",index=False)

In [25]:
df = pd.read_csv("../01_Data/02_Results/result_premium.csv")

In [27]:
df['predicted_premium'] = df['predicted_premium']**0.2

In [28]:
df['predicted_premium']

0        43173.596977
1       108653.835917
2       172065.488056
3       148764.039995
4       144172.203693
            ...      
3254     38092.550199
3255     44259.585552
3256     14802.880589
3257    289381.279423
3258    145743.933599
Name: predicted_premium, Length: 3259, dtype: float64